<a href="https://colab.research.google.com/github/ozdemrburak/Nutrition_Assistant_using_Deep_Learning/blob/main/ResNetRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle'dan veri seti yükleme ve ön işleme

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("siddhantrout/nutrition5k-dataset")

print("Path to dataset files:", path)

In [ ]:
import pickle

images_path = path + "/dish_images.pkl"
with open(images_path, 'rb') as f:
    images = pickle.load(f)

In [ ]:
images.head()

In [ ]:
images.info()

In [ ]:
import pandas as pd
df_i = pd.read_excel(path + "/dish_ingredients.xlsx")
df_i.head()

In [ ]:
# dish_id'ye göre gruplama ve sayısal kolonları toplama, dishes.xslx'de bazı değerler eksikti. Burada malzemelerin calori vs. değerlerini toplayıp doğru değerleri elde ettik.
df_new = df_i.groupby('dish_id')[['grams', 'calories', 'fat', 'carb', 'protein']].sum().reset_index()

print("Yeni DataFrame:")
print(df_new.head())
print(f"\nYeni DataFrame boyutu: {df_new.shape}")
print(f"\nVeri tipleri:")
print(df_new.dtypes)

In [ ]:
images_subset = images[['dish', 'rgb_image']]
df_merged = images_subset.merge(df_new, left_on="dish", right_on="dish_id", how = "inner")
df_merged.info()

In [ ]:
def remove_dish_ids(df_merged, txt_file_path):
    """
    Set operations kullanarak daha hızlı silme işlemi
    """

    # Txt dosyasını oku
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        dish_ids_to_remove = set(line.strip() for line in file if line.strip())

    print(f"Silinecek dish_id sayısı: {len(dish_ids_to_remove)}")

    # Set operations kullanarak filtreleme
    original_size = len(df_merged)
    mask = ~df_merged['dish_id'].isin(dish_ids_to_remove)
    df_cleaned = df_merged[mask].copy()

    final_size = len(df_cleaned)
    print(f"Orijinal: {original_size} -> Temizlenmiş: {final_size}")
    print(f"Silinen: {original_size - final_size} satır")

    return df_cleaned

In [ ]:
dataset_cleaned = remove_dish_ids(df_merged, "exclusion.txt")

In [ ]:
dataset_cleaned.info()

# PyTorch Custom Dataset

In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import io
from sklearn.preprocessing import StandardScaler

In [ ]:
class NutritionDataset(Dataset):
    def __init__(self, df, target_columns, transform=None):
        self.df = df[df['rgb_image'].notna()].reset_index(drop=True)
        self.transform = transform
        self.target_columns = target_columns
        self.targets = self.df[self.target_columns].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_data = row['rgb_image']
        image = Image.open(io.BytesIO(image_data)).convert('RGB')
        target_data = torch.tensor(self.targets[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, target_data


In [ ]:

def create_data_loaders_with_rgb(df, batch_size=32, train_split=0.85, target_columns=None, scaler = None):
  #dataset split
  df_shuffled = df.sample(frac=1).reset_index(drop=True)
  train_size = int(len(df_shuffled) * train_split)
  train_df = df_shuffled[:train_size]
  val_df = df_shuffled[train_size:]
  #scaler
  if target_columns is None:
        target_columns = ['grams', 'calories', 'fat', 'carb', 'protein']
  scaler = StandardScaler()
  train_df[target_columns] = scaler.fit_transform(train_df[target_columns])
  val_df[target_columns]   = scaler.transform(val_df[target_columns])
  #transform
  train_transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.RandomHorizontalFlip(p = 0.4),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  val_transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  #dataset class'ını çağırarak dataset oluşturma
  train_dataset = NutritionDataset(train_df, target_columns, transform= train_transform)
  val_dataset = NutritionDataset(val_df, target_columns, transform= val_transform)
  #dataloaders oluşturma
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader

In [ ]:
train_data, val_data = create_data_loaders_with_rgb(dataset_cleaned)

In [ ]:
# train_data'nın ilk elemanına erişme
train_iter = iter(train_data)
first_train_element = next(train_iter)

# val_data'nın ilk elemanına erişme
val_iter = iter(val_data)
first_val_element = next(val_iter)

print("First element of train_data:")
print(f"Image batch shape: {first_train_element[0].shape}")
print(f"Target batch shape: {first_train_element[1].shape}")

print("\nFirst element of val_data:")
print(f"Image batch shape: {first_val_element[0].shape}")
print(f"Target batch shape: {first_val_element[1].shape}")

# ResNet101 Model Class
ResNet101'in son katmanında fully connected layer 1000 output feature'a sahip. Bunu değiştirmektense bu katmana ek bir RegressionHead yazmak daha doğru diye düşünüyorum. 2 katmanlı bir MLP işimizi görecektir.

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


In [ ]:
class ResNetRegressor(nn.Module):
  def __init__(self, num_outputs = 5):
    super(ResNetRegressor, self).__init__()
    self.backbone = models.resnet101(pretrained = True)
    #self.backbone.fc = nn.Identity()
    self.reg_head = nn.Sequential(
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(in_features=1000, out_features = num_outputs)
    )
  def forward(self, x):
    x = self.backbone(x)
    x= self.reg_head(x)
    return x

In [ ]:
model = ResNetRegressor(num_outputs=5)
print(model)

# Model Eğitim Fonksiyonları

In [ ]:
from tqdm.auto import tqdm
from timeit import default_timer as timer
from torch.optim import optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
from sklearn.metrics import r2_score

def r2_score_torch(y_true, y_pred):
    y_true_np = y_true.detach().cpu().numpy()
    y_pred_np = y_pred.detach().cpu().numpy()
    return r2_score(y_true_np, y_pred_np)


In [ ]:
def rmse_torch(y_true: torch.Tensor, y_pred: torch.Tensor):
    """
    y_true, y_pred: torch.Tensor (batch veya tüm dataset)
    """
    mse = torch.mean((y_true - y_pred) ** 2)
    rmse = torch.sqrt(mse)
    return rmse

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  model.train()
  train_loss = 0.0
  train_r2 = 0.0
  train_rmse = 0.0
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    train_r2 += r2_score_torch(y, y_pred)
    train_rmse += rmse_torch(y, y_pred).item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(dataloader)
  train_r2 = train_r2 / len(dataloader)
  train_rmse = train_rmse / len(dataloader)
  return train_loss, train_r2, train_rmse

In [ ]:
def val_step(model: torch.nn.Module,
             dataloader: torch.utils.data.DataLoader,
             loss_fn: torch.nn.Module,
             device: torch.device):
  model.eval()
  val_loss = 0.0
  val_r2 = 0.0
  val_rmse = 0.0
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      y_pred_logits = model(X)
      loss = loss_fn(y_pred_logits, y)
      val_loss += loss.item()
      val_r2 += r2_score_torch(y, y_pred_logits)
      val_rmse += rmse_torch(y, y_pred_logits).item()

    val_loss = val_loss / len(dataloader)
    val_r2 = val_r2 / len(dataloader)
    val_rmse = val_rmse / len(dataloader)
  return val_loss, val_r2, val_rmse

In [ ]:
def train_loop(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               test_dataloader: torch.utils.data.DataLoader,
               val_dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               loss_fn: torch.nn.Module,
               scheduler: torch.optim.lr_scheduler,
               epochs: int,
               device: torch.device):
  results = {
      "train_loss": [],
      "train_r2": [],
      "train_rmse": [],
      "val_loss": [],
      "val_r2": [],
      "val_rmse": [],
      "optimizer_lr": []
  }
  early_stopper = EarlyStopper(patience=8, min_delta=0.001, save_dir = "/content/checkpoints", )
  for epoch in tqdm(range(epochs)):
    train_loss, train_r2, train_rmse = train_step(model, train_dataloader, loss_fn, optimizer, device) # Corrected argument order
    val_loss, val_r2, val_rmse = val_step(model, val_dataloader, loss_fn, device)
    print(f"Epoch: {epoch+1} | Train loss: {train_loss:.4f} | Train R2: {train_r2:.2f} | Train RMSE: {train_rmse:.4f} | Val loss: {val_loss:.4f} | Val R2: {val_r2:.2f} | Val RMSE: {val_rmse:.4f} | LR: {optimizer.param_groups[0]['lr']} ")
    results["train_loss"].append(train_loss)
    results["train_r2"].append(train_r2)
    results["train_rmse"].append(train_rmse)
    results["val_loss"].append(val_loss)
    results["val_r2"].append(val_r2)
    results["val_rmse"].append(val_rmse)
    results["optimizer_lr"].append(optimizer.param_groups[0]['lr'])
    if early_stopper.early_stop(val_loss, model, epoch):
      print("Early stopping triggered")
      break
    scheduler.step(val_loss)
  return results

# Model Eğitimi

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#source: https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch
import torch
import os

class EarlyStopper:
    def __init__(self, patience=3, min_delta=0.001, save_dir="/content/checkpoints"):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)

    def early_stop(self, validation_loss, model, epoch):
        stop = False
        if validation_loss < self.min_validation_loss - self.min_delta:
            self.min_validation_loss = validation_loss
            self.counter = 0
            save_path = os.path.join(self.save_dir, f"best_model.pth")
            torch.save(model.state_dict(), save_path)
            print(f"Model saved at {save_path}")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                stop = True
        return stop


In [ ]:
NUM_EPOCHS = 75
model = ResNetRegressor(num_outputs=5)
#model.load_state_dict(torch.load("model_state_dict.pth"))
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
loss_fn = nn.L1Loss()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = 0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)
start_time = timer()
model_results = train_loop(model = model,
                           train_dataloader = train_data,
                           test_dataloader = val_data,
                           val_dataloader = val_data,
                           optimizer = optimizer,
                           loss_fn = loss_fn,
                           scheduler = scheduler,
                           epochs = NUM_EPOCHS,
                           device = device)

In [ ]:
model_state_dict = torch.save(model.state_dict(), "model_state_dict.pth")

In [ ]:
import pickle

with open("model_results.pkl", "wb") as f:
    pickle.dump(model_results, f)

print("model_results_1_50.pkl dosyası başarıyla kaydedildi.")

In [ ]:
import torch
import torch.nn.functional as F

model.eval()
with torch.inference_mode():
    X, y = next(iter(val_data))
    X, y = X.to(device), y.float().to(device)
    y_pred = model(X)

# Per-sample MSE ve MAE
mse_per_sample = F.mse_loss(y_pred, y, reduction='none').mean(dim=1)  # [batch]
mae_per_sample = F.l1_loss(y_pred, y, reduction='none').mean(dim=1)

print("Batch MSE mean:", mse_per_sample.mean().item())
print("Batch MAE mean:", mae_per_sample.mean().item())

# Per-feature errors (averaged over batch)
mse_per_feature = F.mse_loss(y_pred, y, reduction='none').mean(dim=0)  # [5]
mae_per_feature = F.l1_loss(y_pred, y, reduction='none').mean(dim=0)
print("MSE per feature:", mse_per_feature.tolist())
print("MAE per feature:", mae_per_feature.tolist())

# A few example predictions vs targets -> ['grams', 'calories', 'fat', 'carb', 'protein']

for i in range(5):
    print("GT:", y[i].cpu().numpy(), "PRED:", y_pred[i].cpu().numpy())


# Metrik Eğrileri

In [ ]:
import matplotlib.pyplot as plt

# Loss grafiği
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(model_results["train_loss"], label="Train Loss")
plt.plot(model_results["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train & Validation Loss")
plt.legend()
plt.grid(True)

# R² grafiği
plt.subplot(1,2,2)
plt.plot(model_results["train_r2"], label="Train R²")
plt.plot(model_results["val_r2"], label="Val R²")
plt.xlabel("Epoch")
plt.ylabel("R² Score")
plt.title("Train & Validation R²")
plt.legend()
plt.grid(True)

# RMSE grafiği
plt.figure(figsize=(12,5))
plt.plot(model_results["train_rmse"], label="Train RMSE")
plt.plot(model_results["val_rmse"], label="Val RMSE")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("Train & Validation RMSE")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
